In [1]:
%%capture
#Installing the requirements that Google Colab doesn't have
!pip install timm 
!pip install wandb --quiet
!pip install pytorch-lightning

In [2]:
#All of our imports
import numpy as np
import pandas as pd

import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.optim import AdamW
from torch.nn import functional as F
from torch.optim.lr_scheduler import CosineAnnealingLR

import torchvision
from torchvision import transforms as T
from torchvision.io import read_image

from torchmetrics import R2Score

import timm

from tqdm import tqdm_notebook as tqdm

import pytorch_lightning as pl
from pytorch_lightning.utilities.seed import seed_everything
from pytorch_lightning.core.lightning import LightningModule
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks import progress
from pytorch_lightning.callbacks.progress import TQDMProgressBar
from pytorch_lightning.callbacks import LearningRateMonitor

import sklearn
from sklearn.model_selection import StratifiedKFold

import wandb

In [3]:
#Need torchvision .8+ for some image reading functionality
torchvision.__version__

'0.11.1'

In [4]:
#Reading in the data
df = pd.read_csv(f'/home/ubuntu/Final_CNN_Dataframe.csv')

In [5]:
df

,cell_id,date,SWE,mean_inversed_swe,mean_local_swe,median_local_swe,max_local_swe,min_local_swe,mean_local_elevation,median_local_elevation,...,MOD10A1_SnowCover,MOD10A1_Albedo,MOD10A1_NDSI,MYD10A1_SnowCover,MYD10A1_Albedo,MYD10A1_NDSI,copernicus_filelocations,sentinel1_filelocation,sentinel2a_filelocation,sentinel2b_filelocation
0,ASO_50M_SWE_USCARC_8,2017-07-17,48.561301,4.697841,4.685963,1.381988,18.482857,0.573913,2834.64000,2880.36,...,0.761684,0.367174,0.766883,0.814966,0.345533,0.818492,/home/ubuntu/SnowData/CopernicusData/ASO_50M_S...,/home/ubuntu/SnowData/Sen1_Data_poly/ASO_50M_S...,/home/ubuntu/SnowData/Sen2_DataA_poly/ASO_50M_...,/home/ubuntu/SnowData/Sen2_DataB_poly/ASO_50M_...
1,ASO_50M_SWE_USCARC_9,2017-07-17,33.924218,4.736198,4.685963,1.381988,18.482857,0.573913,2834.64000,2880.36,...,0.666698,0.326389,0.669003,0.790095,0.339398,0.792235,/home/ubuntu/SnowData/CopernicusData/ASO_50M_S...,/home/ubuntu/SnowData/Sen1_Data_poly/ASO_50M_S...,/home/ubuntu/SnowData/Sen2_DataA_poly/ASO_50M_...,/home/ubuntu/SnowData/Sen2_DataB_poly/ASO_50M_...
2,ASO_50M_SWE_USCARC_26,2017-07-17,31.831418,5.221427,4.685963,1.381988,18.482857,0.573913,2834.64000,2880.36,...,0.717794,0.387605,0.721743,0.660893,0.377931,0.665706,/home/ubuntu/SnowData/CopernicusData/ASO_50M_S...,/home/ubuntu/SnowData/Sen1_Data_poly/ASO_50M_S...,/home/ubuntu/SnowData/Sen2_DataA_poly/ASO_50M_...,/home/ubuntu/SnowData/Sen2_DataB_poly/ASO_50M_...
3,ASO_50M_SWE_USCARC_27,2017-07-17,41.723133,5.322858,4.685963,1.381988,18.482857,0.573913,2834.64000,2880.36,...,0.758141,0.388365,0.761231,0.754647,0.370129,0.758559,/home/ubuntu/SnowData/CopernicusData/ASO_50M_S...,/home/ubuntu/SnowData/Sen1_Data_poly/ASO_50M_S...,/home/ubuntu/SnowData/Sen2_DataA_poly/ASO_50M_...,/home/ubuntu/SnowData/Sen2_DataB_poly/ASO_50M_...
4,ASO_50M_SWE_USCARC_28,2017-07-17,42.693821,5.377803,4.685963,1.381988,18.482857,0.573913,2834.64000,2880.36,...,0.732553,0.393211,0.737283,0.753537,0.346098,0.757798,/home/ubuntu/SnowData/CopernicusData/ASO_50M_S...,/home/ubuntu/SnowData/Sen1_Data_poly/ASO_50M_S...,/home/ubuntu/SnowData/Sen2_DataA_poly/ASO_50M_...,/home/ubuntu/SnowData/Sen2_DataB_poly/ASO_50M_...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97636,ASO_50M_SWE_USCALB_79,2018-06-01,0.474991,2.126230,1.906877,1.888571,3.574384,0.000000,2808.36624,2880.36,...,0.050713,0.056434,0.044083,0.018015,0.023185,-0.121343,/home/ubuntu/SnowData/CopernicusData/ASO_50M_S...,/home/ubuntu/SnowData/Sen1_Data_poly/ASO_50M_S...,/home/ubuntu/SnowData/Sen2_DataA_poly/ASO_50M_...,/home/ubuntu/SnowData/Sen2_DataB_poly/ASO_50M_...
97637,ASO_50M_SWE_USCALB_87,2018-06-01,1.828016,2.115175,1.906877,1.888571,3.574384,0.000000,2808.36624,2880.36,...,0.028636,0.053653,-0.007782,0.000000,0.000000,-0.069880,/home/ubuntu/SnowData/CopernicusData/ASO_50M_S...,/home/ubuntu/SnowData/Sen1_Data_poly/ASO_50M_S...,/home/ubuntu/SnowData/Sen2_DataA_poly/ASO_50M_...,/home/ubuntu/SnowData/Sen2_DataB_poly/ASO_50M_...
97638,ASO_50M_SWE_USCALB_88,2018-06-01,0.114890,2.125414,1.906877,1.888571,3.574384,0.000000,2808.36624,2880.36,...,0.143382,0.132695,0.066765,0.009245,0.012961,-0.126452,/home/ubuntu/SnowData/CopernicusData/ASO_50M_S...,/home/ubuntu/SnowData/Sen1_Data_poly/ASO_50M_S...,/home/ubuntu/SnowData/Sen2_DataA_poly/ASO_50M_...,/home/ubuntu/SnowData/Sen2_DataB_poly/ASO_50M_...
97639,ASO_50M_SWE_USCALB_89,2018-06-01,2.170014,2.135945,1.906877,1.888571,3.574384,0.000000,2808.36624,2880.36,...,0.115417,0.143560,0.091611,0.010301,0.015503,-0.018298,/home/ubuntu/SnowData/CopernicusData/ASO_50M_S...,/home/ubuntu/SnowData/Sen1_Data_poly/ASO_50M_S...,/home/ubuntu/SnowData/Sen2_DataA_poly/ASO_50M_...,/home/ubuntu/SnowData/Sen2_DataB_poly/ASO_50M_...


In [6]:
#Get cell_id - geometry pairing and merge to df
t = pd.read_csv(f'/home/ubuntu/traindf_allregions.csv',usecols=['cell_id','date','geometry'])

df= df.merge(t,how='left',on=['cell_id','date'])

In [7]:
#Designating which columns are our metadata
feature_cols = [col for col in df.columns 
                if col not in 
                ['cell_id', 'date', 'geometry','MOD10A1_filelocations', 'MYD10A1_filelocations', 
                 'copernicus_filelocations', 'SWE','sentinel1_filelocation','sentinel2a_filelocation',
                 'sentinel2b_filelocation','SWE_Scaled'
                 ,'mean_inversed_swe', 'mean_local_swe', 'median_local_swe', 'max_local_swe', 'min_local_swe',
                 'mean_local_elevation', 'median_local_elevation', 'max_local_elevation', 'min_local_elevation']]

#Min max scaling the meta data
scaler = sklearn.preprocessing.MinMaxScaler()
df[feature_cols] =  scaler.fit_transform(df[feature_cols])

#We will create a separate scaler for the targets so that we can transform them back and forth
target_scaler = sklearn.preprocessing.MinMaxScaler()
target_scaler.fit(np.array(df['SWE']).reshape(-1, 1))
df['SWE_Scaled'] = target_scaler.transform(np.array(df['SWE']).reshape(-1, 1))

tabluar_columns = len(feature_cols)

In [8]:
#Tmp drop two rows until better solution i s found
df = df.drop([56955,82314])
df.reset_index(inplace=True,drop=True)

In [9]:
df

,cell_id,date,SWE,mean_inversed_swe,mean_local_swe,median_local_swe,max_local_swe,min_local_swe,mean_local_elevation,median_local_elevation,...,MOD10A1_NDSI,MYD10A1_SnowCover,MYD10A1_Albedo,MYD10A1_NDSI,copernicus_filelocations,sentinel1_filelocation,sentinel2a_filelocation,sentinel2b_filelocation,geometry,SWE_Scaled
0,ASO_50M_SWE_USCARC_8,2017-07-17,48.561301,4.697841,4.685963,1.381988,18.482857,0.573913,2834.64000,2880.36,...,0.873086,0.842185,0.355794,0.896260,/home/ubuntu/SnowData/CopernicusData/ASO_50M_S...,/home/ubuntu/SnowData/Sen1_Data_poly/ASO_50M_S...,/home/ubuntu/SnowData/Sen2_DataA_poly/ASO_50M_...,/home/ubuntu/SnowData/Sen2_DataB_poly/ASO_50M_...,POLYGON ((-119.27063272296992 37.7439075502243...,0.339387
1,ASO_50M_SWE_USCARC_9,2017-07-17,33.924218,4.736198,4.685963,1.381988,18.482857,0.573913,2834.64000,2880.36,...,0.807323,0.816483,0.349477,0.878423,/home/ubuntu/SnowData/CopernicusData/ASO_50M_S...,/home/ubuntu/SnowData/Sen1_Data_poly/ASO_50M_S...,/home/ubuntu/SnowData/Sen2_DataA_poly/ASO_50M_...,/home/ubuntu/SnowData/Sen2_DataB_poly/ASO_50M_...,POLYGON ((-119.27035750488221 37.7349013266373...,0.237091
2,ASO_50M_SWE_USCARC_26,2017-07-17,31.831418,5.221427,4.685963,1.381988,18.482857,0.573913,2834.64000,2880.36,...,0.842758,0.682966,0.389155,0.792468,/home/ubuntu/SnowData/CopernicusData/ASO_50M_S...,/home/ubuntu/SnowData/Sen1_Data_poly/ASO_50M_S...,/home/ubuntu/SnowData/Sen2_DataA_poly/ASO_50M_...,/home/ubuntu/SnowData/Sen2_DataB_poly/ASO_50M_...,POLYGON ((-119.25983912243692 37.7621381423023...,0.222465
3,ASO_50M_SWE_USCARC_27,2017-07-17,41.723133,5.322858,4.685963,1.381988,18.482857,0.573913,2834.64000,2880.36,...,0.869289,0.779852,0.381121,0.855546,/home/ubuntu/SnowData/CopernicusData/ASO_50M_S...,/home/ubuntu/SnowData/Sen1_Data_poly/ASO_50M_S...,/home/ubuntu/SnowData/Sen2_DataA_poly/ASO_50M_...,/home/ubuntu/SnowData/Sen2_DataB_poly/ASO_50M_...,POLYGON ((-119.25956503213258 37.7531318793437...,0.291596
4,ASO_50M_SWE_USCARC_28,2017-07-17,42.693821,5.377803,4.685963,1.381988,18.482857,0.573913,2834.64000,2880.36,...,0.853199,0.778704,0.356376,0.855029,/home/ubuntu/SnowData/CopernicusData/ASO_50M_S...,/home/ubuntu/SnowData/Sen1_Data_poly/ASO_50M_S...,/home/ubuntu/SnowData/Sen2_DataA_poly/ASO_50M_...,/home/ubuntu/SnowData/Sen2_DataB_poly/ASO_50M_...,POLYGON ((-119.2592910638845 37.74412560089217...,0.298380
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97634,ASO_50M_SWE_USCALB_79,2018-06-01,0.474991,2.126230,1.906877,1.888571,3.574384,0.000000,2808.36624,2880.36,...,0.387460,0.018616,0.023873,0.257797,/home/ubuntu/SnowData/CopernicusData/ASO_50M_S...,/home/ubuntu/SnowData/Sen1_Data_poly/ASO_50M_S...,/home/ubuntu/SnowData/Sen2_DataA_poly/ASO_50M_...,/home/ubuntu/SnowData/Sen2_DataB_poly/ASO_50M_...,POLYGON ((-118.99236337285141 37.5873385237014...,0.003320
97635,ASO_50M_SWE_USCALB_87,2018-06-01,1.828016,2.115175,1.906877,1.888571,3.574384,0.000000,2808.36624,2880.36,...,0.352613,0.000000,0.000000,0.292758,/home/ubuntu/SnowData/CopernicusData/ASO_50M_S...,/home/ubuntu/SnowData/Sen1_Data_poly/ASO_50M_S...,/home/ubuntu/SnowData/Sen2_DataA_poly/ASO_50M_...,/home/ubuntu/SnowData/Sen2_DataB_poly/ASO_50M_...,POLYGON ((-118.9815213549945 37.60554539534471...,0.012776
97636,ASO_50M_SWE_USCALB_88,2018-06-01,0.114890,2.125414,1.906877,1.888571,3.574384,0.000000,2808.36624,2880.36,...,0.402699,0.009553,0.013346,0.254326,/home/ubuntu/SnowData/CopernicusData/ASO_50M_S...,/home/ubuntu/SnowData/Sen1_Data_poly/ASO_50M_S...,/home/ubuntu/SnowData/Sen2_DataA_poly/ASO_50M_...,/home/ubuntu/SnowData/Sen2_DataB_poly/ASO_50M_...,POLYGON ((-118.98128235871765 37.5965372685042...,0.000803
97637,ASO_50M_SWE_USCALB_89,2018-06-01,2.170014,2.135945,1.906877,1.888571,3.574384,0.000000,2808.36624,2880.36,...,0.419393,0.010645,0.015963,0.327799,/home/ubuntu/SnowData/CopernicusData/ASO_50M_S...,/home/ubuntu/SnowData/Sen1_Data_poly/ASO_50M_S...,/home/ubuntu/SnowData/Sen2_DataA_poly/ASO_50M_...,/home/ubuntu/Sn

In [10]:
df[(df['cell_id']=='ASO_50M_SWE_USCAKC_1755')& (df['date']== '2019-06-11')]

,cell_id,date,SWE,mean_inversed_swe,mean_local_swe,median_local_swe,max_local_swe,min_local_swe,mean_local_elevation,median_local_elevation,...,MOD10A1_NDSI,MYD10A1_SnowCover,MYD10A1_Albedo,MYD10A1_NDSI,copernicus_filelocations,sentinel1_filelocation,sentinel2a_filelocation,sentinel2b_filelocation,geometry,SWE_Scaled
90555,ASO_50M_SWE_USCAKC_1755,2019-06-11,39.395702,26.869506,21.684,25.11,36.0,0.82,2916.936,2956.56,...,0.866739,0.0,0.0,0.807571,/home/ubuntu/SnowData/CopernicusData/ASO_50M_S...,/home/ubuntu/SnowData/Sen1_Data_poly/ASO_50M_S...,/home/ubuntu/SnowData/Sen2_DataA_poly/ASO_50M_...,/home/ubuntu/SnowData/Sen2_DataB_poly/ASO_50M_...,POLYGON ((-118.89230654533912 37.1906678512196...,0.27533


In [12]:
#Join weather data
#a = pd.read_csv('/home/ubuntu/SnowData/Unique_CellIDs_byDate__ASO_50M_SWE_USCALB__with_HRRR_TMP_surface_12h.csv',index_col=0)
#b = pd.read_csv('/home/ubuntu/SnowData/Unique_CellIDs_byDate__ASO_50M_SWE_USCALB__with_HRRR_PRATE_surface_12h.csv',index_col=0)
#weather = pd.merge(a, b,  how='left', on = ['index','cell_id','geometry','date','month_year'])
weather = pd.read_csv('/home/ubuntu/SnowData/GRIDMET_Weather_Data_V3.csv')

In [13]:
#weather[weather.duplicated(subset=['cell_id', 'date'],keep=False)].sort_values(['cell_id', 'date']).head(50)

In [14]:
#Drop any entries with duplicate cell_id and date combination
#weather = weather.drop_duplicates(subset=['cell_id', 'date'], keep='first')

In [15]:
#weather[(weather['cell_id']=='ASO_50M_SWE_USCARC_8')&(weather['geometry']==a)&(weather['date']=='2017-07-17')]

In [16]:
year = []
for i in df['date'].values:
    if pd.to_datetime(i).strftime('%Y') in year:
        continue
    else:
        year.append(pd.to_datetime(i).strftime('%Y'))

In [17]:
year

['2017', '2019', '2016', '2018']

In [18]:
df[df['cell_id'] == 'ASO_50M_SWE_USCATB_2267']

,cell_id,date,SWE,mean_inversed_swe,mean_local_swe,median_local_swe,max_local_swe,min_local_swe,mean_local_elevation,median_local_elevation,...,MOD10A1_NDSI,MYD10A1_SnowCover,MYD10A1_Albedo,MYD10A1_NDSI,copernicus_filelocations,sentinel1_filelocation,sentinel2a_filelocation,sentinel2b_filelocation,geometry,SWE_Scaled
7536,ASO_50M_SWE_USCATB_2267,2017-08-16,0.000000,2.035490,2.174248,0.426335,9.612795,0.000000,2322.454088,2331.72,...,0.413564,0.000000,0.000000,0.433309,/home/ubuntu/SnowData/CopernicusData/ASO_50M_S...,/home/ubuntu/SnowData/Sen1_Data_poly/ASO_50M_S...,/home/ubuntu/SnowData/Sen2_DataA_poly/ASO_50M_...,/home/ubuntu/SnowData/Sen2_DataB_poly/ASO_50M_...,POLYGON ((-119.53128775945261 37.9910717933107...,0.000000
13982,ASO_50M_SWE_USCATB_2267,2016-04-26,29.657914,24.970221,25.838000,32.650000,55.800000,0.000000,2441.326088,2331.72,...,0.570611,0.000000,0.000000,0.767115,/home/ubuntu/SnowData/CopernicusData/ASO_50M_S...,/home/ubuntu/SnowData/Sen1_Data_poly/ASO_50M_S...,/home/ubuntu/SnowData/Sen2_DataA_poly/ASO_50M_...,/home/ubuntu/SnowData/Sen2_DataB_poly/ASO_50M_...,POLYGON ((-119.53128775945261 37.9910717933107...,0.207274
15107,ASO_50M_SWE_USCATB_2267,2017-01-29,39.044215,32.621191,29.960000,33.354286,42.864286,15.700000,2322.454088,2331.72,...,0.872834,0.612198,0.348205,0.846093,/home/ubuntu/SnowData/CopernicusData/ASO_50M_S...,/home/ubuntu/SnowData/Sen1_Data_poly/ASO_50M_S...,/home/ubuntu/SnowData/Sen2_DataA_poly/ASO_50M_...,/home/ubuntu/SnowData/Sen2_DataB_poly/ASO_50M_...,POLYGON ((-119.53128775945261 37.9910717933107...,0.272874
24903,ASO_50M_SWE_USCATB_2267,2017-07-27,0.000000,2.186704,2.390149,0.351801,10.991677,0.000000,2322.454088,2331.72,...,0.176000,0.000000,0.000000,0.045450,/home/ubuntu/SnowData/CopernicusData/ASO_50M_S...,/home/ubuntu/SnowData/Sen1_Data_poly/ASO_50M_S...,/home/ubuntu/SnowData/Sen2_DataA_poly/ASO_50M_...,/home/ubuntu/SnowData/Sen2_DataB_poly/ASO_50M_...,POLYGON ((-119.53128775945261 37.9910717933107...,0.000000
26029,ASO_50M_SWE_USCATB_2267,2016-03-26,34.362943,9.228537,10.195728,1.927619,44.894286,0.971429,2322.454088,2331.72,...,0.865329,0.763864,0.313841,0.848192,/home/ubuntu/SnowData/CopernicusData/ASO_50M_S...,/home/ubuntu/SnowData/Sen1_Data_poly/ASO_50M_S...,/home/ubuntu/SnowData/Sen2_DataA_poly/ASO_50M_...,/home/ubuntu/SnowData/Sen2_DataB_poly/ASO_50M_...,POLYGON ((-119.53128775945261 37.9910717933107...,0.240157
33788,ASO_50M_SWE_USCATB_2267,2018-04-23,20.860187,10.663129,9.042857,10.668571,18.158571,0.014286,2322.454088,2331.72,...,0.780527,0.501567,0.255780,0.757579,/home/ubuntu/SnowData/CopernicusData/ASO_50M_S...,/home/ubuntu/SnowData/Sen1_Data_poly/ASO_50M_S...,/home/ubuntu/SnowData/Sen2_DataA_poly/ASO_50M_...,/home/ubuntu/SnowData/Sen2_DataB_poly/ASO_50M_...,POLYGON ((-119.53128775945261 37.9910717933107...,0.145789
35492,ASO_50M_SWE_USCATB_2267,2016-04-07,25.710933,8.163772,8.914702,1.961143,40.367143,0.000000,2322.454088,2331.72,...,0.676421,0.000000,0.000000,0.621020,/home/ubuntu/SnowData/CopernicusData/ASO_50M_S...,/home/ubuntu/SnowData/Sen1_Data_poly/ASO_50M_S...,/home/ubuntu/SnowData/Sen2_DataA_poly/ASO_50M_...,/home/ubuntu/SnowData/Sen2_DataB_poly/ASO_50M_...,POLYGON ((-119.53128775945261 37.9910717933107...,0.179690
38484,ASO_50M_SWE_USCATB_2267,2018-05-28,2.257307,0.075943,0.066000,0.000000,0.240000,0.000000,2322.454088,2331.72,...,0.270596,0.073802,0.074757,0.395168,/home/ubuntu/SnowData/CopernicusData/ASO_50M_S...,/home/ubuntu/SnowData/Sen1_Data_poly/ASO_50M_S...,/home/ubuntu/SnowData/Sen2_DataA_poly/ASO_50M_...,/home/ubuntu/SnowData/Sen2_DataB_poly/ASO_50M_...,POLYGON ((-119.53128775945261 37.9910717933107...,0.015776
54559,ASO_50M_SWE_USCATB_2267,2016-05-09,26.732759,5.312869,5.628157,2.050540,23.761429,0.000000,2322.454088,2331.72,...,0.853569,0.704663,0.257716,0.806836,/home/ubuntu/SnowData/CopernicusData/ASO_50M_S...,/home/ubuntu/SnowData/Sen1_Data_poly/ASO_50M_S...,/home/ubuntu/SnowData/Sen2_DataA_poly/ASO_50M_...,/home/ubuntu/SnowData/Sen2_DataB_poly/ASO_

In [19]:
class args:
    #Overall Args
    folder_name = "/home/ubuntu/SnowData"

    #Keep track of features used in wandb
    features = feature_cols
    lstm_features = ['precip_daily','wind_dir_avg','temp_min','temp_max','wind_vel']

    #Setting the number of CPU workers we are using
    num_workers = 12

    #Setting the seed so we can replicate
    seed = 1212

    #Toggle for whether or not we want our model pretrained on imagenet
    pretrained = True

    #Next we pick the model name with the appropriate shape, img size and output
    model_name1 = 'mixnet_s'
    model_shape1 = 1536
    model_name2 = 'tf_efficientnet_b2_ns'
    model_shape2 = 1408 #768 for swin small 1536 for swin large 1792 for efficientnet b4 768 for cait-m-36
    imagesize = 224
    num_classes = 1
    img_channels = 3
    
    dropout1 = .1
    dropout2 = .3

    #LSTM variables
    lstm_hidden = 256
    lstm_layers = 2
    lstm_seqlen = 10

    #Training Args
    train_batch_size = 24
    val_batch_size = 24
    test_batch_size = 24

    #Max epochs and number of folds
    max_epochs = 100
    n_splits = 2

    #Optimizer and Scheduler args
    loss = 'nn.BCEWithLogitsLoss'
    lr = 4e-4
    warmup_epochs = 5
    weight_decay = 4e-6
    eta_min = 0.000001
    n_accumulate = 1
    T_0 = 50
    T_max = 2000

    #Callback args
    #Minimum number amount of improvement to not trigger patience
    min_delta = 0.0
    #Number of epochs in a row to wait for improvement
    patience = 30

#Dataloader Args
loaderargs = {'num_workers' : args.num_workers, 'pin_memory': False, 'drop_last': False}
device = torch.device("cuda:0")

seed_everything(args.seed)

Global seed set to 1212


1212

In [20]:
#Datasets are how pytorch knows how to read in the data
class SWEDataset(torch.utils.data.Dataset):
    def __init__(self, df,ts, test = False, seq_len = 10):
        self.df = df
        self.seq_len = seq_len
        #First we must specify the path to the images
        #self.MOD10A1_file_names = df['MOD10A1_filelocations'].values
        #self.MYD10A1_file_names = df['MYD10A1_filelocations'].values
        self.copernicus_file_names = df['copernicus_filelocations'].values
        self.sentinel1_file_names = df['sentinel1_filelocation'].values
        self.sentinel2a_file_names = df['sentinel2a_filelocation'].values
        self.sentinel2b_file_names = df['sentinel2b_filelocation'].values
        #Variables to query time series and output
        self.cell_id = df['cell_id'].values
        self.date = df['date'].values
        self.geometry = df['geometry'].values
        ts['date'] = pd.to_datetime(ts['date'])
        self.timeseries = ts
        #The only transform we want to do right now is the resizing
        self._transform = T.Resize(size= (args.imagesize, args.imagesize))
        #We specify the tabular feature columns
        self.meta = df[feature_cols].values
        #Now we specify the targets
        self.targets = df['SWE_Scaled'].values
        #Finally we specify if this is training or test
        self.test = test
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        #Get the image, scale it to between 0-1 and resize it
        copernicus_img_path = self.copernicus_file_names[index]
        copernicus_img = read_image(copernicus_img_path,mode = torchvision.io.image.ImageReadMode.RGB) / 255
        copernicus_img = self._transform(copernicus_img)
        
        sentinel1_img_path = self.sentinel1_file_names[index]
        sentinel1_img = read_image(sentinel1_img_path, mode = torchvision.io.image.ImageReadMode.RGB) / 255
        sentinel1_img = self._transform(sentinel1_img)
        
        sentinel2a_img_path = self.sentinel2a_file_names[index]
        sentinel2a_img = read_image(sentinel2a_img_path, mode = torchvision.io.image.ImageReadMode.RGB) / 255
        sentinel2a_img = self._transform(sentinel2a_img)
        
        sentinel2b_img_path = self.sentinel2b_file_names[index]
        sentinel2b_img = read_image(sentinel2b_img_path, mode = torchvision.io.image.ImageReadMode.RGB) / 255
        sentinel2b_img = self._transform(sentinel2b_img)

        #Pull from weather data and generate time-series
        #Variables = precip_daily|wind_dir_avg|temp_min|temp_max|wind_vel
        date_range = pd.date_range(end=self.date[index], periods=self.seq_len)
        
        sequence = self.timeseries.loc[(self.timeseries['cell_id'] == self.cell_id[index]) 
                                       & (self.timeseries['geometry'] == self.geometry[index]) 
                                       & (self.timeseries.date.isin(date_range))
                                      ].sort_values('date')
        #fill na values
        sequence = sequence.fillna(-1)
        if list(sequence.shape) != [10,8]:
            missing_dates = [[self.cell_id[index],-1,i,-1,-1,-1,-1,-1] for i in date_range if i not in list(sequence.date)]
            missing_dates = pd.DataFrame(data = missing_dates,columns=['cell_id','geometry','date','precip_daily','wind_dir_avg','temp_min','temp_max','wind_vel'])
            sequence = pd.concat([sequence,missing_dates],axis=0,ignore_index=True).sort_values('date')
        #Drop non-tabular data columns
        sequence.drop(['cell_id','geometry','date'],axis=1,inplace=True)
        
        ts = torch.tensor(sequence.values.tolist(),dtype=torch.float32)

        
        #Pull in the features for our batch
        meta = self.meta[index, :]
        
        #Specify the target based on whether this is training or test
        if self.test:
          target = 0
        else:
          target = self.targets[index]
            
        return copernicus_img, sentinel1_img, sentinel2a_img, sentinel2b_img, target, meta , ts

In [21]:
#Pytorch Lightning Requires that the dataset be formatted as a module
class SWEDataModule(pl.LightningDataModule):
    def __init__(self, traindf, valdf,ts,args, loaderargs):
        super().__init__()
        #Import our training and validation set, which we will define later
        self._train_df = traindf
        self._val_df = valdf
        self.ts = ts

        #Makesure we bring in our args so we can use them
        self.args = args
        self.loaderargs = loaderargs

    #Building the datasets
    def __create_dataset(self, train=True):
        if train == 'train':
          return SWEDataset(self._train_df,self.ts)
        else:
          return SWEDataset(self._val_df, self.ts)

    #Using the datasets to return a dataloader
    def train_dataloader(self):
        SWE_train = self.__create_dataset("train")
        return DataLoader(SWE_train, **self.loaderargs, batch_size=self.args.train_batch_size)

    def val_dataloader(self):
        SWE_val = self.__create_dataset("val")
        return DataLoader(SWE_val, **self.loaderargs, batch_size=self.args.val_batch_size)

In [22]:
def get_default_transforms():
    transform = {
        "train": T.Compose(
            [
                #T.RandomHorizontalFlip(),
                #T.RandomVerticalFlip(),
                #T.RandomAffine(15, translate=(0.1, 0.1), scale=(0.9, 1.1)),
                T.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3),
                T.ConvertImageDtype(torch.float),
                T.Normalize(mean = (0.485, 0.456, 0.406), 
                            std = (0.229, 0.224, 0.225))
                
            ]
        ),
        "val": T.Compose(
            [
                T.ConvertImageDtype(torch.float),
                T.Normalize(mean = (0.485, 0.456, 0.406), 
                            std = (0.229, 0.224, 0.225))
            ]
        ),
    }
    return transform
  

def mixup(x1: torch.Tensor, x2: torch.Tensor, x3: torch.Tensor,x4: torch.Tensor,
          y: torch.Tensor, 
          z = torch.Tensor, alpha: float = 1.0):
    assert alpha > 0, "alpha should be larger than 0"
    assert x1.size(0) > 1, "Mixup cannot be applied to a single instance."

    lam = np.random.beta(alpha, alpha)
    rand_index = torch.randperm(x1.size()[0])
    mixed_x1 = lam * x1 + (1 - lam) * x1[rand_index, :]
    mixed_x2 = lam * x2 + (1 - lam) * x2[rand_index, :]
    mixed_x3 = lam * x3 + (1 - lam) * x3[rand_index, :]
    mixed_x4 = lam * x4 + (1 - lam) * x4[rand_index, :]
    mixed_meta = lam * z + (1 - lam) * z[rand_index, :]
    target_a, target_b = y, y[rand_index]
    return mixed_x1,mixed_x2,mixed_x3, mixed_x4,mixed_meta, target_a, target_b,  lam

In [25]:
#Model
class CNNLSTM(LightningModule):
    def __init__(self):
        super().__init__()
        self.args = args
        self.scaler = target_scaler
        self.tabular_columns = tabluar_columns
        self._criterion = eval(self.args.loss)()
        self.transform = get_default_transforms()
        
        self.relu = nn.ReLU()
        self.dropout1 = nn.Dropout(p=args.dropout1)
        self.dropout2 = nn.Dropout(p=args.dropout2)
                
        #Tracking
        self.trainr2 = R2Score()
        self.valr2 = R2Score()
        
        #Image Models
        self.model1 = timm.create_model(args.model_name1, 
                                       pretrained=args.pretrained, 
                                       num_classes=0,
                                       in_chans = 3,
                                       #global_pool=''
                                       )
        self.model2 = timm.create_model(args.model_name2, 
                                       pretrained=args.pretrained, 
                                       num_classes=0,
                                       in_chans = 3,
                                       #global_pool=''
                                       )
        self.model3 = timm.create_model(args.model_name2, 
                                       pretrained=args.pretrained, 
                                       num_classes=0,
                                       in_chans = 3,
                                       #global_pool=''
                                       )
        self.model4 = timm.create_model(args.model_name2, 
                                       pretrained=args.pretrained, 
                                       num_classes=0,
                                       in_chans = 3,
                                       #global_pool=''
                                       )
        #LSTM
        self.lstm = nn.LSTM(input_size = 5,
                            hidden_size = args.lstm_hidden,
                            num_layers = args.lstm_layers,
                            batch_first=True,dropout=.1)
        #Possible multiple LSTM layers?
        #self.lstm = nn.LSTM(input_size = args.lstm_hidden,
        #                    hidden_size = self.hidden_size,
        ##                    num_layers = args.lstm_layers,
        #                    batch_first=True,dropout=.1)
        
        #Linear regression layer
        self.linear1 = nn.Linear(8326,1024)
        self.linear2 = nn.Linear(1024,256)
        self.linear3 = nn.Linear(256,args.num_classes)
    
        
    def forward(self,features1,features2,features3,features4,meta,ts):
        
        
        
        #Image Convolution
        #Image Models
        features1 = self.model1(features1)                 
        features1 = self.relu(features1)
        features1 = self.dropout1(features1)
        
        features2 = self.model2(features2)                 
        features2 = self.relu(features2)
        features2 = self.dropout1(features2)
        
        features3 = self.model3(features3)                 
        features3 = self.relu(features3)
        features3 = self.dropout1(features3)
        
        features4 = self.model4(features4)                 
        features4 = self.relu(features4)
        features4 = self.dropout1(features4)
        

        #LSTM
        batch_size, seq_len, feature_len = ts.size()
        # Initialize hidden state with zeros
        
        h_0 = torch.zeros(args.lstm_layers, batch_size, args.lstm_hidden,requires_grad=True).cuda()
        c_0 = torch.zeros(args.lstm_layers, batch_size, args.lstm_hidden,requires_grad=True).cuda()
        
        f_ts, (final_hidden,final_cell) = self.lstm(ts, (h_0,c_0))
        f_ts = f_ts.contiguous().view(batch_size,-1)
        
        #*************************************************************
        #Concatenate meta and image features
        features = torch.cat([features1,features2,features3,features4,f_ts,meta],dim=1)
        #*************************************************************
        
        #Linear
        features = self.linear1(features)
        features = self.relu(features)
        features = self.dropout2(features)
        
        features = self.linear2(features)
        features = self.relu(features)
        features = self.dropout2(features)
        
        output = self.linear3(features)
        return output
    
###I DIDN"T MIX UP TS data
    def __share_step(self, batch, mode):
        copernicus_img, sentinel1_img, sentinel2a_img, sentinel2b_img, labels, meta,ts = batch
        labels = labels.float()
        meta = meta.float()
        ts = ts.float()
        copernicus_img = self.transform[mode](copernicus_img)
        sentinel1_img = self.transform[mode](sentinel1_img)
        sentinel2a_img = self.transform[mode](sentinel2a_img)
        sentinel2b_img = self.transform[mode](sentinel2b_img)

        rand_index = torch.rand(1)[0]
        
        #This is a mixup function
        if rand_index < 0.5 and mode == 'train':
            copernicus_mixed,sentinel1_mixed,sentinel2a_mixed,sentinel2b_mixed, mixed_meta, target_a, target_b, lam = mixup(
                                                          copernicus_img,sentinel1_img,sentinel2a_img,sentinel2b_img,
                                                          labels, meta, alpha=0.5)
            logits = self.forward(copernicus_mixed,sentinel1_mixed,sentinel2a_mixed,sentinel2b_mixed, mixed_meta,ts).squeeze(1)
            loss = self._criterion(logits, target_a) * lam + \
                (1 - lam) * self._criterion(logits, target_b)

        else:  
          logits = self.forward(copernicus_img,sentinel1_img,sentinel2a_img,sentinel2b_img, meta,ts).squeeze(1)
          loss = self._criterion(logits, labels)

        pred = torch.from_numpy(self.scaler \
            .inverse_transform(np.array(logits.sigmoid().detach().cpu()) \
            .reshape(-1, 1)))
        labels = torch.from_numpy(self.scaler \
            .inverse_transform(np.array(labels.detach().cpu()) \
            .reshape(-1, 1)))
        
        '''
        #This is random noise
        elif rand_index > 0.8 and mode == 'train':
            images = images + (torch.randn(images.size(0),3,args.imagesize,args.imagesize, 
                                           dtype = torch.float, device = device)*10)/100
            logits = self.forward(images, meta).squeeze(1)
            loss = self._criterion(logits, labels)
        '''

        return loss, pred, labels

    def training_step(self, batch, batch_idx):
        loss, pred, labels = self.__share_step(batch, 'train')
        self.trainr2(pred.cuda(),labels.cuda())
        self.log('train_loss', loss, on_step=True, on_epoch=True, logger=True)
        return {'loss': loss, 'pred': pred, 'labels': labels}
    
    def validation_step(self, batch, batch_idx):
        loss, pred, labels = self.__share_step(batch, 'val')
        self.valr2(pred.cuda(),labels.cuda())
        self.log('val_loss', loss, on_step=False, on_epoch=True, prog_bar=True)
        return {'pred': pred, 'labels': labels}

    def training_epoch_end(self, outputs):
        self.log('train_r2_epoch',self.trainr2)
        self.__share_epoch_end(outputs, 'train')

    def validation_epoch_end(self, outputs):
        self.log('val_r2_epoch',self.valr2)
        self.__share_epoch_end(outputs, 'val')

    def __share_epoch_end(self, outputs, mode):
        preds = []
        labels = []
        for out in outputs:
            pred, label = out['pred'], out['labels']
            preds.append(pred)
            labels.append(label)
        preds = torch.cat(preds)
        labels = torch.cat(labels)
        metrics = torch.sqrt(((labels - preds) ** 2).mean())
        self.log(f'{mode}_RMSE', metrics)    


    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr=args.lr, weight_decay = args.weight_decay)
        
        return {
        "optimizer": optimizer,
        "lr_scheduler": {
            "scheduler": CosineAnnealingLR(optimizer, T_max = args.T_max, eta_min= args.eta_min),
            "interval": "step",
            "monitor": "train_loss",
            "frequency": 1}
            }

In [26]:
#Not doing kfold, instead separating by year
traindf = df[(pd.to_datetime(df['date']).dt.year == int('2019'))|
            (pd.to_datetime(df['date']).dt.year == int('2018'))].copy().reset_index(drop=True)
valdf = df[(pd.to_datetime(df['date']).dt.year == int('2016'))|
            (pd.to_datetime(df['date']).dt.year == int('2017'))].copy().reset_index(drop=True)

model = CNNLSTM()

modelname = 'cnn-lstm-gridv3-fillna-nosnotel-v2

#Callbacks
early_stop_callback = EarlyStopping(monitor="val_RMSE", min_delta=args.min_delta, patience=args.patience, 
                                    verbose=False, mode="min")
progressbar = TQDMProgressBar(refresh_rate = 10)
checkpoint_callback = ModelCheckpoint(dirpath='/home/ubuntu/snowcap/weights', 
                                      filename= f"{modelname}_best_weights", save_top_k=1, monitor="val_RMSE")
lr_monitor = LearningRateMonitor(logging_interval='step')

#Initialize wandb()
#wandb.init(name=modelname,project = "cnn-lstm_Modeling", entity = "snowcastshowdown", job_type='train')

#Log model parameters into wandb (args variable dictionary)
args_dict = dict(args.__dict__)
#pop out non-json-able variables
for key in ['__module__','__dict__','__weakref__','__doc__']:
    args_dict.pop(key,None)
#wandb.config.update(args_dict)


#wandb_logger = WandbLogger(log_model = 'all')

#wandb_logger.watch(model)

trainer = pl.Trainer(max_epochs=args.max_epochs, 
                    gpus=1, 
#                    logger=wandb_logger,
                    callbacks=[early_stop_callback, 
                                progressbar, 
                                checkpoint_callback,
                                lr_monitor])

SWE_Datamodule = SWEDataModule(traindf, valdf, weather, args = args, loaderargs = loaderargs)

trainer.fit(model, SWE_Datamodule)

#wandb.finish()

del model
torch.cuda.empty_cache()

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name       | Type              | Params
--------------------------------------------------
0  | _criterion | BCEWithLogitsLoss | 0     
1  | relu       | ReLU              | 0     
2  | dropout1   | Dropout           | 0     
3  | dropout2   | Dropout           | 0     
4  | trainr2    | R2Score           | 0     
5  | valr2      | R2Score           | 0     
6  | model1     | EfficientNet      | 2.6 M 
7  | model2     | EfficientNet      | 7.7 M 
8  | model3     | EfficientNet      | 7.7 M 
9  | model4     | EfficientNet      | 7.7 M 
10 | lstm       | LSTM              | 795 K 
11 | linear1    | Linear            | 6.6 M 
12 | linear2    | Linear            | 262 K 
13 | linear3    | Linear            | 257   
--------------------------------------------------
33.3 M    Trainable params
0         Non-trainable params
33.3 M    Tot

Validation sanity check: 0it [00:00, ?it/s]

RuntimeError: mat1 and mat2 shapes cannot be multiplied (24x8326 and 6406x1024)

In [24]:
wandb.finish()